In [ ]:
from flask import Flask, request, jsonify
import os
import datetime

app = Flask(__name__)

# Define the folder where images will be saved
IMAGE_FOLDER = "img"

# Ensure the folder exists
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

@app.route('/upload', methods=['POST'])
@app.route('/img', methods=['POST'])  # Added '/img' route
def upload_image():
    """Endpoint to receive images from ESP32-S3 and save them."""
    if request.data:  # Image sent as raw binary
        # Generate a timestamped filename
        filename = f"captured_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        file_path = os.path.join(IMAGE_FOLDER, filename)

        # Save the image
        with open(file_path, "wb") as f:
            f.write(request.data)

        return jsonify({"status": "success", "filename": filename, "path": file_path}), 200

    return jsonify({"status": "error", "message": "No image received"}), 400

@app.route('/list_images', methods=['GET'])
def list_images():
    """Endpoint to list all saved images."""
    images = os.listdir(IMAGE_FOLDER)
    return jsonify({"images": images}), 200

@app.route('/')
def home():
    """Homepage to indicate the server is running."""
    return "ESP32-S3 Flask Image Server is running!", 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.100:5000
Press CTRL+C to quit


In [ ]:
from flask import Flask, request, jsonify
import os
import datetime

app = Flask(__name__)

# Define the folder where images will be saved
IMAGE_FOLDER = "img"

def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

ensure_folder_exists(IMAGE_FOLDER)

# Store the latest command received
latest_command = "S"  # Default to stop

@app.route('/upload', methods=['POST'])
def upload_image():
    """Endpoint to receive images from ESP32-S3 and save them."""
    if request.data:  # Image sent as raw binary
        filename = f"captured_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        file_path = os.path.join(IMAGE_FOLDER, filename)

        with open(file_path, "wb") as f:
            f.write(request.data)

        print(f"✅ Image saved: {file_path}")
        return jsonify({"status": "success", "filename": filename, "path": file_path}), 200
    
    return jsonify({"status": "error", "message": "No image received"}), 400

@app.route('/list_images', methods=['GET'])
def list_images():
    """Endpoint to list all saved images."""
    images = os.listdir(IMAGE_FOLDER)
    return jsonify({"images": images}), 200

# ✅ API to set motor control command
@app.route('/set_command', methods=['POST'])
def set_command():
    global current_command
    data = request.get_json()

    if not data or "command" not in data:
        return jsonify({"error": "Missing command"}), 400

    command = data["command"].upper()

    if command not in ["F", "B", "S", "R", "L"]:
        return jsonify({"error": "Invalid command"}), 400

    current_command = command  # ✅ Store the latest command
    print(f"✅ Motor Command updated: {current_command}")

    return jsonify({"message": f"Motor Command set to {current_command}"}), 200

# ✅ API for ESP32 to fetch the latest motor control command
@app.route('/get_command', methods=['GET'])
def get_command():
    return jsonify({"command": current_command}), 200
@app.route('/')
def home():
    """Homepage to indicate the server is running."""
    return jsonify({"status": "running", "message": "ESP32-S3 Flask Image & Motor Server is live!"}), 200

if __name__ == '__main__':
    print("🚀 Flask server is starting...")
    app.run(host='0.0.0.0', port=5000)


🚀 Flask server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.100:5000
Press CTRL+C to quit
